In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
with open('/home/becode/Downloads/page_1.csv') as f:
    new_dict1 = pd.read_csv('/home/becode/Downloads/page_1.csv')
new_dict1 = new_dict1[new_dict1['Type'] == 'house']
new_dict1= new_dict1[new_dict1['Price'] < 800000]
new_dict1.dropna(subset = ["Price", "Living area", 'Kitchen type', 'Type',  "Bedrooms", 'Surface of the plot'], axis=0, inplace = True)
#new_dict1.drop(["How many fireplaces?", "ID", 'Locality', 'Type'], axis=1, inplace = True)
new_dict = new_dict1[["Price", "Living area", "Bedrooms", 'Surface of the plot']]
new_dict.head()

,Price,Living area,Bedrooms,Surface of the plot
2629,299000.0,138.0,2.0,882.0
2630,240000.0,88.0,2.0,192.0
2631,270000.0,158.0,3.0,295.0
2632,209000.0,125.0,3.0,299.0
2633,249000.0,201.0,2.0,95.0


In [2]:
print("There are {} rows of data".format(len(new_dict)))
print(new_dict.dtypes)

There are 4184 rows of data
Price                  float64
Living area            float64
Bedrooms               float64
Surface of the plot    float64
dtype: object


In [3]:
#new_dict = new_dict[new_dict.loc[:]!=0].dropna()
print("There are {} rows of data".format(len(new_dict)))
new_dict.head()

There are 4184 rows of data


,Price,Living area,Bedrooms,Surface of the plot
2629,299000.0,138.0,2.0,882.0
2630,240000.0,88.0,2.0,192.0
2631,270000.0,158.0,3.0,295.0
2632,209000.0,125.0,3.0,299.0
2633,249000.0,201.0,2.0,95.0


In [4]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
with open('/home/becode/Downloads/page_1.csv') as f:
    new_dict1 = pd.read_csv('/home/becode/Downloads/page_1.csv')
new_dict1 = new_dict1[new_dict1['Type'] == 'house']
new_dict1= new_dict1[new_dict1['Price'] < 800000]
new_dict1.dropna(subset = ["Price", "Living area", 'Kitchen type', 'Type',  "Bedrooms", 'Surface of the plot'], axis=0, inplace = True)
#new_dict1.drop(["How many fireplaces?", "ID", 'Locality', 'Type'], axis=1, inplace = True)
new_dict = new_dict1[["Price", "Living area", "Bedrooms", 'Surface of the plot']]
# select the numeric columns and drop the y column 'Price'
numeric_features = new_dict.select_dtypes(include=['float64'])
numeric_features.drop('Price', axis=1, inplace=True)
numeric_transformer = Pipeline(steps=[('poly',PolynomialFeatures(degree =2)), ('scaler', StandardScaler())])

# fit the pipeline to train a model
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features.columns.values)])
rg_model = RandomForestRegressor()        
pre = Pipeline(steps=[('preprocessor', preprocessor)])
model = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', rg_model)])

X = new_dict.drop(['Price'], axis = 1)
y = new_dict['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, shuffle = False, random_state = 0)
#print(X_train.iloc[0])
model.fit(X, y)
model.score(X, y)

rg_model.fit(X, y)
print(rg_model.score(X, y))


0.8124043470408098


In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
with open('/home/becode/Downloads/page_1.csv') as f:
    new_dict1 = pd.read_csv('/home/becode/Downloads/page_1.csv')
new_dict1 = new_dict1[new_dict1['Type'] == 'house']
new_dict1= new_dict1[new_dict1['Price'] < 800000]
new_dict1.dropna(subset = ["Price", "Living area", 'Kitchen type', 'Type',  "Bedrooms", 'Surface of the plot'], axis=0, inplace = True)
new_dict = new_dict1[["Price", "Living area", "Bedrooms", 'Surface of the plot']]
rg_model = RandomForestRegressor()        

X = new_dict.drop(['Price'], axis = 1).values
y = new_dict['Price'].values
#model.fit(X, y)
rg_model.fit(X, y)


RandomForestRegressor()

In [15]:
from pickle import dump, load
import pickle as pk
from skops.io import dumps, loads
with open("filename.skops", 'wb') as pickle_file:
    pk.dump(rg_model, pickle_file)
#dump(rg_model, "filename.skops")



In [21]:
X_t = [[132, 2, 882]]
with open('filename.skops', 'rb') as f: 
    clf = load(f)
res1 = clf.predict(X_t)
print('Predicted price ' + str(int(res1))+ '€')
#X_t = {'Living area':67.0, 'Bedrooms':2.0, 'Surface of the plot': 67.0}
res = rg_model.predict(X_t)
print('Predicted price ' + str(int(res))+ '€')

Predicted price 292327€
Predicted price 292327€


In [8]:
from flask import Flask, redirect, render_template, url_for, request
app = Flask(__name__)

@app.route('/success/<name>')
def success(name):
    return 'Your estimated house price (after calculation) %s' % name

@app.route('/home',methods = ['POST', 'GET'])
def login():
    if request.method == 'POST':
        salary = request.form['salary']
        tax =  request.form['tax']
        bonus =  request.form['bonus']
        X_t = [[salary, tax, bonus]]

        clf = load('filename.joblib') 
        res1 = clf.predict(X_t)
        res = 'Predicted price ' + str(int(res1))+ '€'
        return redirect(url_for('success', name = res ))
    else:
        salary = request.args.get('salary')
        tax = request.args.get('tax')
        bonus = request.args.get('bonus')
        #total_salary = salary + bonus  - tax
       # return redirect(url_for('success',name = user))
        return render_template('home.html', name=[salary, tax, bonus])
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [31/Jan/2023 12:39:26] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [31/Jan/2023 12:39:34] "POST /home HTTP/1.1" 302 -
127.0.0.1 - - [31/Jan/2023 12:39:34] "GET /success/Predicted%20price%20290378€ HTTP/1.1" 200 -
